In [1]:
#from arcgis.gis import GIS
# from arcgis.raster.analytics import copy_raster
import arcpy
#print(os.environ['PATH'])
import os
import datetime as dt
import xml.dom.minidom as DOM

HOMEdir = os.getcwd()
DATAdir = HOMEdir + r'\data'
RESULTSdir = HOMEdir + r'\results'
SHAREdir = HOMEdir + r'\share'
REPROJdir = HOMEdir + r'\reproject'

In [2]:
arcGISproProj = r"C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\indonezja\DiengPlateauSharing\DiengPlateauSharing.aprx"

In [4]:
def uploadAGOL(indexName):
    arcpy.SignInToPortal("https://www.arcgis.com", "zuluzi", "Jem987dzem!")
    print('Start publishing : ' + indexName)
    os.chdir(SHAREdir)
# this layer (index) was uploaded before
    if os.path.isfile('.\\' + indexName + '.sd'):
        outdir = SHAREdir
        service_name = indexName + str(dt.datetime.today().strftime('%Y%m%d'))
        sddraft_filename = service_name + ".sddraft"
        sddraft_output_filename = os.path.join(outdir, sddraft_filename)
        sddraft_temp_output_filename = os.path.join(outdir, service_name + 'temp.sddraft')
        sd_filename = service_name + ".sd"
        sd_output_filename = os.path.join(outdir, sd_filename)

        # Reference map to publish
        aprx = arcpy.mp.ArcGISProject(arcGISproProj)
        m = aprx.listMaps(indexName)[0]

        # Create TileSharingDraft and set metadata and portal folder properties
        server_type = "HOSTING_SERVER"
        sddraft = m.getWebLayerSharingDraft(server_type, "TILE", service_name)
        sddraft.credits = ""
        sddraft.description = "Layer containg raster values for " + indexName + "."
        sddraft.summary = ""
        sddraft.tags = "Water Quality, Sentinel-2, " + indexName
        sddraft.useLimitations = ""
        sddraft.portalFolder = "DiengPlateau"
        sddraft.overwriteExistingService = True
        sddraft.exportToSDDraft(sddraft_temp_output_filename)
        
        # Modyf minScale and maxScale in sddraft file
        doc = DOM.parse(sddraft_temp_output_filename)
        prop = doc.getElementsByTagName('ConfigurationProperties')[0]
        configProps = doc.getElementsByTagName('ConfigurationProperties')[0]
        propArray = configProps.firstChild
        propSets = propArray.childNodes
        
        for propSet in propSets:
            keyValues = propSet.childNodes
            for keyValue in keyValues:
                if keyValue.tagName == 'Key':
                    if keyValue.firstChild.data == "minScale":
                        # set min scale
                        keyValue.nextSibling.firstChild.data = "1150000.000000"
            for keyValue in keyValues:
                if keyValue.tagName == 'Key':
                    if keyValue.firstChild.data == "maxScale":
                        # set max scale
                        keyValue.nextSibling.firstChild.data = "5000.000000"

        f = open(sddraft_output_filename, 'w')
        doc.writexml(f)
        f.close()
        
        # Stage
        print("    Start Staging")
        arcpy.StageService_server(sddraft_output_filename, sd_output_filename)

        # Upload
        print("    Start Uploading")
        arcpy.UploadServiceDefinition_server(sd_output_filename, server_type)
        
        # Creates and updates tiles in an existing web tile layer cache
        print("    Start Creating Tiles")
        input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
        arcpy.ManageMapServerCacheTiles_server(input_service, [200000,50000,10000], "RECREATE_ALL_TILES")
        
        # Replace prod layer with temp
        print("    Start Replacing Layers")
        main_service = repr(r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + indexName + r'/MapServer')
        input_service = repr(r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer')
        archive_serviceName = indexName + r'archive' + str(dt.datetime.today().strftime('%Y%m%d'))

        arcpy.server.ReplaceWebLayer(main_service, archive_serviceName, input_service)
        
        os.remove(sddraft_output_filename)
        os.remove(sd_output_filename)
        os.remove(sddraft_temp_output_filename)
# this layer (index) is uploaded for the first time
    else:
        outdir = SHAREdir
        service_name = indexName
        sddraft_filename = service_name + ".sddraft"
        sddraft_output_filename = os.path.join(outdir, sddraft_filename)
        sddraft_temp_output_filename = os.path.join(outdir, service_name + 'temp.sddraft')
        sd_filename = service_name + ".sd"
        sd_output_filename = os.path.join(outdir, sd_filename)

        # Reference map to publish
        aprx = arcpy.mp.ArcGISProject(arcGISproProj)
        m = aprx.listMaps(indexName)[0]

        # Create TileSharingDraft and set metadata and portal folder properties
        server_type = "HOSTING_SERVER"
        sddraft = m.getWebLayerSharingDraft(server_type, "TILE", service_name)
        sddraft.credits = ""
        sddraft.description = "Layer containg raster values for " + indexName + "."
        sddraft.summary = ""
        sddraft.tags = "Water Quality, Sentinel-2, " + indexName
        sddraft.useLimitations = ""
        sddraft.portalFolder = "DiengPlateau"
        sddraft.overwriteExistingService = True
        sddraft.exportToSDDraft(sddraft_temp_output_filename)
        
        # Modyf minScale and maxScale in sddraft file
        doc = DOM.parse(sddraft_temp_output_filename)
        prop = doc.getElementsByTagName('ConfigurationProperties')[0]
        configProps = doc.getElementsByTagName('ConfigurationProperties')[0]
        propArray = configProps.firstChild
        propSets = propArray.childNodes
        
        for propSet in propSets:
            keyValues = propSet.childNodes
            for keyValue in keyValues:
                if keyValue.tagName == 'Key':
                    if keyValue.firstChild.data == "minScale":
                        # set min scale
                        keyValue.nextSibling.firstChild.data = "1150000.000000"
            for keyValue in keyValues:
                if keyValue.tagName == 'Key':
                    if keyValue.firstChild.data == "maxScale":
                        # set max scale
                        keyValue.nextSibling.firstChild.data = "5000.000000"

        f = open(sddraft_output_filename, 'w')
        doc.writexml(f)
        f.close()
        
        # Stage
        print("    Start Staging")
        arcpy.StageService_server(sddraft_output_filename, sd_output_filename)

        # Upload
        print("    Start Uploading")
        arcpy.UploadServiceDefinition_server(sd_output_filename, server_type)
        
        # Creates and updates tiles in an existing web tile layer cache
        print("    Start Creating Tiles")
        input_service = repr(r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer')
        arcpy.ManageMapServerCacheTiles_server(input_service, [150000,50000,10000], "RECREATE_ALL_TILES")
        
    print('Finish publishing : ' + indexName)
    os.chdir(HOMEdir)


In [5]:
for file in os.listdir(SHAREdir):
    if file.endswith('cya_merged_masked.tif'):
        uploadAGOL(file.split('_')[0] + '_DP')

Start publishing : cya_DP
    Start Staging
    Start Uploading
    Start Creating Tiles
Finish publishing : cya_DP


In [5]:
service_name = 'chla' + str(dt.datetime.today().strftime('%Y%m%d'))
input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
arcpy.ManageMapServerCacheTiles_server(input_service, [500000,50000,10000], "RECREATE_ALL_TILES")

<Result 'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/admin/services/chla20220102/MapServer/jobs/b90d57b28a6bbd95'>

In [11]:
print("    Start Replacing Layers")
main_service = repr(r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + r'CDOM_DP' + r'/MapServer')
input_service = repr(r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + r'CDOM_DP20220331' + r'/MapServer')
archive_serviceName = r'CDOM_DP' + r'archive' + str(dt.datetime.today().strftime('%Y%m%d'))

arcpy.server.ReplaceWebLayer(main_service, archive_serviceName, input_service)

    Start Replacing Layers


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Target Layer: Dataset 'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/CDOM_DP/MapServer' does not exist or is not supported
Failed to execute (ReplaceWebLayer).


In [7]:
service_name = 'chla' + str(dt.datetime.today().strftime('%Y%m%d'))

main_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + 'chla' + r'/MapServer'
input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
archive_serviceName = 'chla' + r'archive' + str(dt.datetime.today().strftime('%Y%m%d'))

arcpy.ReplaceWebLayer_server(main_service, archive_serviceName, input_service)

os.remove(sddraft_output_filename)
os.remove(sd_output_filename)
os.remove(sddraft_temp_output_filename)

NameError: name 'sddraft_output_filename' is not defined

In [20]:
arcpy.SignInToPortal("https://www.arcgis.com", "zuluzi", "Jem987dzem!")

{'token': 'OGONfUk-Fzn0YgIFd5fCOMsEvVYX1MYn_BPOj4ZGh3ZubA_zllJqHHvfv9QVp4-h-9Xxo_JorcWJpEqpDckBB7wHyT1cfULxwsSHvD5Out7ziyN1kqUavGlZhQIa6biABQouHQ5tB55CctIR_Xhw7KPxtsbjXnBKrl2fM7wgSvE7fjV-r-rzbolCBcjU7chsqMQgKQxoOFhDiTC1xGoBeNiIDWjPbk7uRLyIb14gAS0.',
 'referer': 'http://www.esri.com/AGO/28666298-5D8D-443D-8638-AB5BF3746B16',
 'expires': 1639878919}

In [24]:
input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + ' chlatemp20211212' + r'/MapServer'
# arcpy.server.DeleteMapServerCache(input_service)
arcpy.ManageMapServerCacheTiles_server(input_service, [500000,50000,10000],"DELETE_TILES")

ExecuteError: ERROR 160235: There is not enough storage space to complete the operation.
Failed to execute (ManageMapServerCacheTiles).


In [25]:
arcpy.server.DeleteMapServerCache(input_service)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 001265: Admin or publisher connection required.
Failed to execute (DeleteMapServerCache).


In [3]:
for file in os.listdir(SHAREdir):
    if file.endswith('turbidity_merged_masked.tif'):
        uploadAGOL(file.split('_')[0])

Start publishing : turbidity
    Start Staging
    Start Uploading
    Start Creating Tiles
    Start Replacing Layers
Finish publishing : turbidity


In [7]:
arcpy.SignInToPortal("https://www.arcgis.com", "zuluzi", "Jem987dzem!")
# arcpy.SignOutFromPortal_server()

service_name = 'turbidity20211218'
indexName = 'turbidity'

# print("    Start Creating Tiles")
# input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
# arcpy.ManageMapServerCacheTiles_server(input_service, [500000,50000,10000], "RECREATE_ALL_TILES")

# Replace prod layer with temp
print("    Start Replacing Layers")
main_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + indexName + r'/MapServer'
input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
archive_serviceName = indexName + r'archive' + str(dt.datetime.today().strftime('%Y%m%d'))

arcpy.ReplaceWebLayer_server(main_service, archive_serviceName, input_service)

    Start Replacing Layers


<Result 'https://www.arcgis.com/home/item.html?id=e70aaae655ab406594691609f94aa633'>

In [30]:
print(os.path.join(SHAREdir,'CDOM'+'temp'))

C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz\share\CDOM\temp


In [3]:
import inspect
import xml.dom.minidom as DOM

arcpy.SignInToPortal("https://www.arcgis.com", "zuluzi", "Jem987dzem!")
outdir = HOMEdir
service_name = 'CDOMtest'
sddraft_filename = service_name + ".sddraft"
sddraft_output_filename = os.path.join(outdir, sddraft_filename)
sd_filename = service_name + ".sd"
sd_output_filename = os.path.join(outdir, sd_filename)

arcGISproProj = r"C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz\GINinz_ArcGISPro_project\GINinz_ArcGISPro_project.aprx"
aprx = arcpy.mp.ArcGISProject(arcGISproProj)
m = aprx.listMaps('CDOM')[0]

# Create TileSharingDraft and set metadata and portal folder properties
server_type = "HOSTING_SERVER"
sddraft = m.getWebLayerSharingDraft(server_type, "TILE", service_name)
sddraft.credits = ""
sddraft.description = "Layer containg raster values for CDOM."
sddraft.summary = ""
sddraft.tags = "Water Quality, Sentinel-2, CDOM"
sddraft.useLimitations = "For internal use only"
sddraft.portalFolder = "GINinz_prod"
sddraft.overwriteExistingService = True

sddraft.exportToSDDraft(sddraft_output_filename)

# # Stage
# print("    Start Staging")
# arcpy.StageService_server(sddraft_output_filename, sd_output_filename)

# print(sddraft)

In [17]:
doc = DOM.parse(sddraft_output_filename)
# configProps = doc.getElementsByTagName('Info')[0]
# propArray = configProps.firstChild
# propSets = propArray.childNodes
# for propSet in propSets:
#     keyValues = propSet.childNodes
#     for keyValue in keyValues:
#         print(keyValue.tagName)
#         if keyValue.tagName == 'Key':
#             print(keyValue.firstChild.data)
#             # if keyValue.firstChild.data == "WebCapabilities":
#             #     # Defaults are Map,Query,Data
#             #     keyValue.nextSibling.firstChild.data = "Map,Data"

# typeNames = doc.getElementsByTagName('TypeName')
# for typeName in typeNames:
#     # Get the TypeName to enable
#     if typeName.firstChild.data == "FeatureServer":
#         extension = typeName.parentNode
#         for extElement in extension.childNodes:
#             if extElement.tagName == 'Info':
#                 for propSet in extElement.childNodes:
#                     for prop in propSet.childNodes:
#                         for prop1 in prop.childNodes:
#                             if prop1.tagName == "Key":
#                                 print(prop1.firstChild.data)
#                                 if prop1.firstChild.data == 'WebCapabilities':
#                                     # Defaults are Query,Create,Update,Delete,Uploads,Editing
#                                     prop1.nextSibling.firstChild.data = "Create,Sync,Query"

prop = doc.getElementsByTagName('ConfigurationProperties')[0]

configProps = doc.getElementsByTagName('ConfigurationProperties')[0]
propArray = configProps.firstChild
propSets = propArray.childNodes
for propSet in propSets:
    keyValues = propSet.childNodes
    for keyValue in keyValues:
        if keyValue.tagName == 'Key':
            if keyValue.firstChild.data == "minScale":
                # set min scale
                keyValue.nextSibling.firstChild.data = "1200000.000000"
    for keyValue in keyValues:
        if keyValue.tagName == 'Key':
            if keyValue.firstChild.data == "maxScale":
                # set max scale
                keyValue.nextSibling.firstChild.data = "4000.000000"

f = open(HOMEdir + '\\CDOMtest2.sddraft', 'w')
doc.writexml( f )
f.close()

In [18]:
arcpy.StageService_server(HOMEdir + '\\CDOMtest2.sddraft', HOMEdir + '\\CDOMtest2.sd')

<Result 'C:\\Users\\Asus\\OneDrive - Stowarzyszenie IAESTE Polska\\Studia\\GINinzynierka\\GINinz\\CDOMtest2.sd'>

In [8]:
inspect.getmembers(sddraft)

[('__class__', arcpy.sharing.TileSharingDraft),
 ('__delattr__',
  <method-wrapper '__delattr__' of TileSharingDraft object at 0x000001C4A01FFE48>),
 ('__dict__',
  {'_arc_object': <SharingDraftObject at 0x1c4a0b64890>,
   'overwriteExistingService': True,
   'serverType': 'HOSTING_SERVER',
   'serviceType': 'TILE',
   'serviceName': 'CDOMtest',
   'portalFolder': 'GINinz_prod',
   'summary': '',
   'tags': 'Water Quality, Sentinel-2, CDOM',
   'description': 'Layer containg raster values for CDOM.',
   'credits': '',
   'useLimitations': 'For internal use only',
   'offline': False,
   'server_type': 'HOSTING_SERVER',
   'service_type': 'TILE',
   '__id': 4,
   '__m_path': 'CIMPATH=map/map.xml',
   '__premise_path': 'C:\\Users\\Asus\\OneDrive - Stowarzyszenie IAESTE Polska\\Studia\\GINinzynierka\\GINinz\\GINinz_ArcGISPro_project\\GINinz_ArcGISPro_project.aprx',
   '__llist': [],
   '__tlist': []}),
 ('__dir__', <function TileSharingDraft.__dir__()>),
 ('__doc__', None),
 ('__eq__',
  

In [18]:
print(str(dt.datetime.today().strftime('%Y%m%d')))

20211201


In [18]:
test = arcpy.da.Walk()
print(test)

ValueError: Workspace is not set

In [16]:
# Sign in to portal
arcpy.SignInToPortal("https://www.arcgis.com", "zuluzi", "Jem987dzem!")

{'token': '6849qFpRa24r6Kd_yV-SuAguZ65jxuEBBI-yifbCB3ll5jEcAkAzhplrXhAvZHIVkFaHl2RPMxYGKnLQgOGwV7i9f4CZOwDzAHKpKIn9l2iU5jkB2VRh5wTltr1iSa4J8YhyeJW2h6w7TgF33sS-uva0qx3n8LTL6n-Lm50B-2kcvA3fdfgka4DQ-CpbeKYoL6yNIq3km-0KafYoIHPuV41Y1uPA0A0DUhZRrccZDK8.',
 'referer': 'http://www.esri.com/AGO/322923C3-A5AE-4E81-8B13-142E2F99CDE4',
 'expires': 1638172880}

In [45]:
os.chdir(r'C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz')
print(os.getcwd())

C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz


In [3]:
# Set output file names
outdir = SHAREdir
service_name = "WaterQualityIndexes"
sddraft_filename = service_name + ".sddraft"
sddraft_output_filename = os.path.join(outdir, sddraft_filename)
sd_filename = service_name + ".sd"
sd_output_filename = os.path.join(outdir, sd_filename)

# Reference map to publish
arcGISproProj = r"C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz\GINinz_ArcGISPro_project\GINinz_ArcGISPro_project.aprx"
aprx = arcpy.mp.ArcGISProject(arcGISproProj)
m = aprx.listMaps('Map')[0]

# Create TileSharingDraft and set metadata and portal folder properties
server_type = "HOSTING_SERVER"
sddraft = m.getWebLayerSharingDraft(server_type, "TILE", service_name)
sddraft.credits = ""
sddraft.description = "Layers containg rasters with CDOM, Chl a, Cya, DOC and Turbidity values."
sddraft.summary = "This is summary"
sddraft.tags = "Water Quality, Sentinel-2, CDOM, Chl a, Cya, DOC, Turbidity"
sddraft.useLimitations = "For internal use only"
sddraft.portalFolder = "GINinz_prod"
sddraft.overwriteExistingService = True

In [4]:
# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraft_output_filename)

# Stage Service
print("Start Staging")
arcpy.StageService_server(sddraft_output_filename, sd_output_filename)
print("Finish Staging")


Start Staging
Finish Staging


In [5]:
# Share to portal
#server_type = "HOSTING_SERVER"
#sd_output_filename = r"C:\Users\Asus\OneDrive - Stowarzyszenie IAESTE Polska\Studia\GINinzynierka\GINinz\share\test_v5.sd"
print("Start Uploading")
arcpy.UploadServiceDefinition_server(sd_output_filename, server_type)

print("Finish Publishing")

Start Uploading
Finish Publishing


In [6]:
# Creates and updates tiles in an existing web tile layer cache. 
input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
arcpy.ManageMapServerCacheTiles_server(input_service, [500000], "RECREATE_ALL_TILES")
print("Created Tiles")

Created Tiles


In [ ]:
print(input_service)

In [ ]:
main_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/test_overwrite/MapServer'
input_service = r'https://tiles.arcgis.com/tiles/MzCtPDSne0rpIt7V/arcgis/rest/services/' + service_name + r'/MapServer'
archive_serviceName = r'testOverwrite_archive'

arcpy.ReplaceWebLayer_server(main_service, archive_serviceName, input_service)


In [ ]:
gis = GIS(username="zuluzi", password="Jem987dzem!")

In [ ]:
my_content = gis.content.search(query="")

my_content

In [ ]:
input_raster_path = r"C:\Users\Asus\Documents\CDOM_S2A_MSIL2A_20211024T095101_N0301_R079_T33UYR_20211024T122307.tif"
single_image_layer = copy_raster(input_raster=input_raster_path,
                                 output_name="output_imagery_layer",
                                 gis=gis)

In [ ]:
results = os.listdir(RESULTSdir)
print(RESULTSdir + '\\' + results[0])

In [ ]:
os.environ['PATH'] = r';C:\Program Files\PostgreSQL\14\bin'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'
os.environ['PGUSER'] = 'postgres'
os.environ['PGPASSWORD'] = 'postgres'
os.environ['PGDATABASE'] = 'WQ'

results = os.listdir(RESULTSdir)

conn = psycopg2.connect(database="WQ", user="postgres", host="localhost", password="postgres")
cursor = conn.cursor()
cmds = 'raster2pgsql -t 100x100 "' + RESULTSdir + '\\' + results[0] + '" cdomrast.cdom1 |psql'

sql = "INSERT INTO rasters.cdom VALUES (st_raster(" + RESULTSdir + '\\' + results[0] + ",'compression=LZW'));"

subprocess.call(sql, shell=True)

conn.close()

In [ ]:
conn.close()

In [ ]:
if not products.empty:
    for product in products.iterrows():
        bands = bands2A(product)
for band in bands.keys():
    print(f'{band}: {bands[band]}')
for product in products.iterrows():
    for v1 in product[1].keys():
        print(f'{v1}: {product[1][v1]}')

In [ ]:
try1 = geojson_to_wkt(read_geojson('malopolska.geojson'))

In [ ]:
    srcB03 = rasterio.open(bands['b02_10m'])
    kwargs = srcB03.meta
    print(type(kwargs))
    print(kwargs['crs'])

In [ ]:
#B04 = rasterio.open(bands['b04_10m']).read(1).astype('f4')
#print(B04.shape[1])
#i = 0
#if B04[B04 <= 0] <= 0:
#    i = i+1
#print(i)


In [ ]:
for product in products.iterrows():
    prodBands = bands2A(product)
    CDOM(product[1]['title'], prodBands['B03'],prodBands['B04'])

In [ ]:
src = rasterio.open(RESULTSdir+r"\test123.tif")
kwargs = src.meta
print(kwargs)
srcB03 = rasterio.open(prodBands['B03'])
kwargs = srcB03.meta
print(kwargs)

In [ ]:
os.chdir(DATAdir) # get into data folder
for product in products_df.iterrows():
    try:
        prodBands = bandsDir(product)
        rgb = RGB(prodBands['REDdir'],prodBands['BLUEdir'],prodBands['GREENdir'])
        plt.imshow(rgb, cmap='RdYlGn')
        plt.colorbar()
        plt.title('RGB')
        plt.xlabel('Column #')
        plt.ylabel('Row #')
        os.chdir(DATAdir) # back to data folder
    except NameError:
        print('Error occurred while processing {0}'.format(product[1]['filename']))
os.chdir(HOMEdir) # back to main directory

In [ ]:
# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(prodBands['REDdir']) as src:
    print(type(src.read()[0][0][0]))
    print(src.read()[0][0][0])
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    print(oviews)
    oview = oviews[-1] # let's look at the smallest thumbnail
    print(oview)
    print('Decimation factor= {}'.format(oview))
    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
products_df_sorted = products_df_sorted.head(1)
#print(products_df_sorted.uuid[0])
api.get_product_odata(products_df_sorted.uuid[0], full=True)
#api.download(products_df_sorted.uuid[0])

In [ ]:
odata = api.get_product_odata(products_df_sorted.uuid[0], full=True)
with zipfile.ZipFile(odata['title']+'.zip',"r") as zip_ref:
    zip_ref.extractall()

In [ ]:
#odata = api.get_product_odata(products_df_sorted.uuid[0], full=True)
#print(odata)
odata = api.get_product_odata(products_df_sorted.uuid[0], full=True)
print(odata['title'])
for row in products_df_sorted.head(1):
    print(row)

print(products_df_sorted.head(1).title)
#print(products_df_sorted.head(1))

In [ ]:
for product in products_df.iterrows():
        prodBands = bandsDir(product)
        print(prodBands)